# Hybrid RAG System Development

This notebook covers the step-by-step development of the RAG system.

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('../.env')

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")

assert GROQ_API_KEY, "GROQ_API_KEY not found"
print("Environment keys loaded.")

## 1. Vector Database Setup (Qdrant)

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

if QDRANT_URL and QDRANT_API_KEY:
    client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)
else:
    # Local memory mode for testing
    client = QdrantClient(location=":memory:")

collection_name = "dev_rag_collection"

if not client.collection_exists(collection_name):
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=384, distance=Distance.COSINE),
    )
    print(f"Collection '{collection_name}' created.")
else:
    print(f"Collection '{collection_name}' exists.")

## 2. Ingestion Pipeline

In [ ]:
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# Setup Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Vector Store wrapper
vector_store = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=embeddings,
)

# Example Document
text = """
LangChain is a framework for developing applications powered by language models. 
Qdrant is a vector similarity search engine and vector database.
Streamlit turns data scripts into shareable web apps in minutes.
"""
docs = [Document(page_content=text, metadata={"source": "dummy"})]

# Splitting
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

# Add to Vector Store
vector_store.add_documents(splits)
print(f"Added {len(splits)} chunks to Qdrant.")

## 3. Retrieval and Generation

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

llm = ChatGroq(
    groq_api_key=GROQ_API_KEY,
    model_name="llama3-70b-8192"
)

retriever = vector_store.as_retriever()

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What is Qdrant?"})
print("Answer:", response["answer"])

## 4. (Extension) Graph Knowledge Experiment
Here you can experiment with NetworkX or Neo4j integration.